# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install citipy

Note: you may need to restart the kernel to use updated packages.


In [39]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [40]:
# store the CSV in WeatherPy into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hilo,19.7297,-155.0900,74.25,91,100,6.91,US,1667717337
1,1,rocha,-34.4833,-54.3333,52.52,97,3,4.43,UY,1667717426
2,2,rock sound,24.9000,-76.2000,80.53,71,2,22.03,BS,1667717429
3,3,hualmay,-11.0964,-77.6139,61.16,87,7,8.32,PE,1667717431
4,4,riyadh,24.6877,46.7219,77.14,32,9,14.81,SA,1667717401


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [41]:
# configure gmaps
gmaps.configure(api_key=g_key)

In [42]:
# heatmap of the humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()

# add the heatmap layer to the figure
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
idealWeatherDF = city_data_df.loc[
    (city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80) & \
    (city_data_df["Wind Speed"] < 10) & \
    (city_data_df["Cloudiness"] == 0)
].dropna()

print(len(idealWeatherDF))

idealWeatherDF.head()

17


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,47,cape town,-33.9258,18.4232,73.11,67,0,9.22,ZA,1667717553
76,76,east london,-33.0153,27.9116,72.54,73,0,3.44,ZA,1667717209
96,96,kruisfontein,-34.0033,24.7314,74.88,54,0,7.70,ZA,1667717182
122,122,hojai,26.0000,92.8667,79.95,68,0,0.78,IN,1667717747
152,152,mondlo,-27.9830,30.7177,70.95,50,0,3.18,ZA,1667717824


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [44]:
hotel_df = idealWeatherDF[["City", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

/var/folders/sh/vr7lnsm55jgbyk1sgr07d30m0000gn/T/ipykernel_47439/1945051825.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Lat,Lng,Hotel Name
47,cape town,-33.9258,18.4232,
76,east london,-33.0153,27.9116,
96,kruisfontein,-34.0033,24.7314,
122,hojai,26.0000,92.8667,
152,mondlo,-27.9830,30.7177,
173,bloomingdale,27.8936,-82.2404,
180,avarua,-21.2078,-159.7750,
182,san patricio,28.0170,-97.5169,
217,ballina,-28.8667,153.5667,
265,kidal,18.4411,1.4078,


In [55]:
#set the parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# loop through the dataframe using iterrows()
for index, row in hotel_df.iterrows():
    # get the latitude and longitude from the current row
    lat = row["Lat"]
    lng = row["Lng"]
    
    # use the latitude and longitude to set the locaation value in the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # use the google apis places base url to search
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print the url
    name_address = requests.get(base_url, params=params).json()
    
    #pprint(name_address) # print ["results"][0]["name"]
    
    try:
        # populate the hotel name
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except:
        print(f"Error finding lodging in {row['City']}. Skipping...")
        pass
hotel_df

Error finding lodging in kidal. Skipping...
Error finding lodging in tessalit. Skipping...


,City,Lat,Lng,Hotel Name
47,cape town,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
76,east london,-33.0153,27.9116,Tu Casa
96,kruisfontein,-34.0033,24.7314,Oyster Bay House Rental
122,hojai,26.0000,92.8667,Luxury park hotel
152,mondlo,-27.9830,30.7177,Zungu N N
173,bloomingdale,27.8936,-82.2404,Travel Club for Free
180,avarua,-21.2078,-159.7750,Paradise Inn
182,san patricio,28.0170,-97.5169,"Motel 6 Sinton, TX"
217,ballina,-28.8667,153.5667,Ramada Hotel & Suites by Wyndham Ballina Byron
265,kidal,18.4411,1.4078,


In [57]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [56]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

NameError: name 'hotel_info' is not defined